In [1]:
import numpy as np
import pandas as pd
#import lightgbm as lgb
import xgboost as xgb

from sklearn import  model_selection, linear_model
from sklearn.metrics import mean_squared_log_error, recall_score
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
from matplotlib import pyplot
import seaborn as sns

import os
import matplotlib.pyplot as plt

In [2]:
data_dir = './data/'

## Utils

In [3]:
def rmsle(est, x, y_test):
    """
    Метрика rmsle

    :est: обученный экземпляр модели
    :x: объекты на которых нужно предсказать значение
    :y_test: фактическое значение объектов
    :returns: значение метрики rmsle на выборке x
    """

    predict = est.predict(x)
    predict = [x if x > 0 else 0 for x in predict]
    return np.sqrt(mean_squared_log_error(y_test, predict ))


def regr_score(x_train, y_train, regr, scoring):
    """
    Расчет кроссвалидации и вывод на экран

    :x_train: обучающая выборка
    :y_train: целевое значение
    :regr: экземпляр модели
    :scoring: метрика
    """
    scores = cross_validate(regr, 
                            x_train, 
                            y_train, 
                            scoring=scoring,
                            cv=5, 
                            return_train_score=False)
    
    scores_list = scores[list(scores.keys())[-1]]
    print(scores_list)
    print(f'mean score -- {np.mean(scores_list)}')
    
    
def get_data():
    df_x = pd.read_csv(f'{data_dir}/train.csv')
    df_x = df_x.fillna(-1)
        
    y = df_x['label']
    df_x = df_x.drop(['label', 'status', 'short', 'activity_title', 'title_activity_type',
                      'activity_description', 'title_direction', 'comment_direction'], 
                     axis=1)
    return df_x, y

## Read data

In [ ]:
df = pd.read_csv(f'{data_dir}/train.csv')
df = df.fillna(-1)
df = df.drop(['status', 'short', 'activity_title', 'title_activity_type', 'activity_description', 
                  'title_direction', 'comment_direction'], axis=1)

In [ ]:
sns.pairplot(df)

In [ ]:
df.corr()

In [4]:
df_x, y = get_data()
print(df_x.shape)
print(len(y))
print(df_x.columns)
df_x.head()

(13000, 11)
13000
Index(['id_bet', 'run_id', 'user_id', 'direction_id', 'activity_id',
       'size_max', 'size_min', 'is_educational', 'is_checkin_required',
       'activity_type_id', 'main_competence_id'],
      dtype='object')


,id_bet,run_id,user_id,direction_id,activity_id,size_max,size_min,is_educational,is_checkin_required,activity_type_id,main_competence_id
0,56,790,844,3.0,358,20,10,1,0,5.0,55.0
1,59,135,898,6.0,29,28,5,1,0,5.0,21.0
2,61,182,188,2.0,42,30,12,1,0,5.0,26.0
3,63,182,552,3.0,42,30,12,1,0,5.0,26.0
4,64,951,898,6.0,426,50,10,1,0,5.0,26.0


In [5]:
x_train = np.array(df_x)
print(x_train)
y_train = np.array(y)

[[5.6000e+01 7.9000e+02 8.4400e+02 ... 0.0000e+00 5.0000e+00 5.5000e+01]
 [5.9000e+01 1.3500e+02 8.9800e+02 ... 0.0000e+00 5.0000e+00 2.1000e+01]
 [6.1000e+01 1.8200e+02 1.8800e+02 ... 0.0000e+00 5.0000e+00 2.6000e+01]
 ...
 [1.6160e+04 4.0800e+02 8.3500e+02 ... 0.0000e+00 5.0000e+00 3.3000e+01]
 [1.6161e+04 5.9600e+02 5.3600e+02 ... 0.0000e+00 6.0000e+00 3.5000e+01]
 [1.6162e+04 2.4800e+02 8.3500e+02 ... 0.0000e+00 2.0000e+00 2.0000e+01]]


In [6]:
# Посмотрим, какие есть файлы и положим их в словарь
dfs = {}
for i, x in enumerate(os.listdir(data_dir)):
    file_name = x.split('.')[0]
    print(f'{i} -- {file_name}')
    dfs[file_name] = pd.read_csv(f'{data_dir}/{x}')

0 -- user_role
1 -- competence
2 -- author
3 -- time_slot
4 -- test
5 -- competence_level
6 -- competence_type
7 -- activity_track
8 -- track
9 -- activity_tag
10 -- role
11 -- train
12 -- place
13 -- activity_author
14 -- event
15 -- user_tag
16 -- sample_submission


## Simple prediction
Для начала нужно попробовать самые простые модели, чтобы примерно понимать, какое качество ожидать

In [ ]:
# Наивная модель, где предсказанием является среднее значение, полученное на обучающей выборке
regr = DummyRegressor(strategy='mean')
regr_score(x_train, y_train, regr, rmsle)
print()

# Наивная модель, где предсказанием является медиана, полученная на обучающей выборке
# Эта статистика менее подвержена выбросам, поэтому, возможно, даст лучшее качество
regr = DummyRegressor(strategy='median')
regr_score(x_train, y_train, regr, rmsle)
print()

# Градиентный бустинг
regr = xgb.XGBRegressor()
#regr = lgb.LGBMRegressor()
regr_score(x_train, y_train, regr, rmsle)
print()

# К средних соседей
regr = KNeighborsRegressor()
regr_score(x_train, y_train, regr, rmsle)

Как ни странно, лучшее качество дал алгоритм ближайших соседей.

Теперь можно попробовать поиграться с гиперпараметрами

In [ ]:
regr = xgb.XGBRegressor(n_estimators=68)
regr_score(x_train, y_train, regr, rmsle)
print()

regr = KNeighborsRegressor(n_neighbors=4, weights='distance',  p=1)
regr_score(x_train, y_train, regr, rmsle)

## Drop outliers
Качество улучшилось, но не сильно. Возможно, нужно что то еще придумать.

Можно посмотреть на наш таргет

In [ ]:
pd.Series(sorted(y_train)).plot()

Видно, что есть выбросы и, возможно, они мешают алгоритму уловить настоящую закономерность, поэтому можно ему помочь, удалив выбросы.

Делать это можно несколькими способами, но возьмем самый простой, просто выкинем все значения которые больше 95 перцентиля. Кстати, это значение тоже можно подбирать как гиперпараметр.

In [ ]:
pd.Series(sorted([x for x in y_train if x < np.quantile(y_train, 0.95)])).plot()

Выглядит уже лучше, ставки на 10000 - 40000 уже не участвуют в обучении.

In [ ]:
# Формирование новой выборки без выбросов
x_train = np.array(df_x)
y_train = np.array(y)

x_train = x_train[[True if x < np.quantile(y_train, 0.95) else False for x in y_train]]
y_train = [x for x in y_train if x < np.quantile(y_train, 0.95)]

По аналогии с тем как делали выше, посмотрим качество моделей

In [ ]:
# Наивная модель где предсказанием является среднее значение полученное на обучающей выборке
regr = DummyRegressor(strategy='mean')
regr_score(x_train, y_train, regr, rmsle)
print()

# Наивная модель где предсказанием является медиана полученная на обучающей выборке
# Эта статистика менее подвержена выбросам, поэтому возможно даст лучшее качество
regr = DummyRegressor(strategy='median')
regr_score(x_train, y_train, regr, rmsle)
print()

# Градиентный бустинг 
regr = xgb.XGBRegressor()
regr_score(x_train, y_train, regr, rmsle)
print()

# К средних соседей
regr = KNeighborsRegressor()
regr_score(x_train, y_train, regr, rmsle)

In [ ]:
regr = xgb.XGBRegressor(n_estimators=66, num_leaves=38)
regr_score(x_train, y_train, regr, rmsle)
print()

regr = KNeighborsRegressor(n_neighbors=4, weights='distance',  p=1)
regr_score(x_train, y_train, regr, rmsle)

## Merge
Добавим данные

In [ ]:
x_train = np.array(df_x)
y_train = np.array(y)

# тут происходит мерж исходных данных и дополнительных, которые мы считали в словарь dfs

df_tmp = pd.merge(df_x, dfs['activity_author'].groupby('activity_id').count().reset_index(),  
                  how='left', 
                  left_on='activity_id', right_on='activity_id', 
                  suffixes=('_x', 'activity_author'))

df_tmp = pd.merge(df_tmp, dfs['event'].groupby('run_id').count().reset_index(),  
                  how='left', 
                  left_on='run_id', right_on='run_id', 
                  suffixes=('_x', '_event'))

df_tmp = pd.merge(df_tmp, dfs['user_role'].drop_duplicates('user_id'),
                  how='left', 
                  left_on='user_id', right_on='user_id', 
                  suffixes=('_x', '_event'))

In [ ]:
regr = xgb.XGBRegressor()
regr_score(df_tmp, y_train, regr, rmsle)
print()


regr = KNeighborsRegressor(n_neighbors=4, weights='distance',  p=1)
regr_score(x_train, y_train, regr, rmsle)

In [ ]:
print(df_tmp.shape)
print(len(y))

df_tmp = df_tmp.fillna(-1)
x_train = np.array(df_tmp)
y_train = np.array(y)

x_train = x_train[[True if x < np.quantile(y_train, 0.95) else False for x in y_train]]
y_train = [x for x in y_train if x < np.quantile(y_train, 0.95)]

print(x_train.shape)
print(len(y_train))

In [ ]:
regr = xgb.XGBRegressor(n_estimators=60, num_leaves=39)
regr_score(x_train, y_train, regr, rmsle)
print()


regr = KNeighborsRegressor(n_neighbors=4, weights='distance',  p=1)
regr_score(x_train, y_train, regr, rmsle)

Качество не улучшилось, но это значит нужно чуть глубже капнуть в дополнительные признаки.

Можно посмотреть что важно в данных по мнению бустинга

In [ ]:
regr = xgb.XGBRegressor()
regr.fit(x_train, y_train)
pyplot.figure(figsize=(20,10))
pyplot.bar(df_tmp.columns, regr.feature_importances_)
pyplot.show()

Самыми важными признакми являются user_id и id_bet

Можно попробовать посмотреть то же самое, заэнкодив признаки

## Predict

Сделаем сабмит

In [7]:
df_tmp = pd.merge(df_x, dfs['activity_author'].groupby('activity_id').count().reset_index(),  
                  how='left', 
                  left_on='activity_id', right_on='activity_id', 
                  suffixes=('_x', 'activity_author'))

df_tmp = pd.merge(df_tmp, dfs['event'].groupby('run_id').count().reset_index(),  
                  how='left', 
                  left_on='run_id', right_on='run_id', 
                  suffixes=('_x', '_event'))

df_tmp = pd.merge(df_tmp, dfs['user_role'].drop_duplicates('user_id'),
                  how='left', 
                  left_on='user_id', right_on='user_id', 
                  suffixes=('_x', '_event'))

In [8]:
df_x_test = pd.read_csv(f'{data_dir}/test.csv')
df_x_test = df_x_test.fillna(-1)
df_x_test = df_x_test.drop(['short', 'activity_title', 'title_activity_type',
            'activity_description', 'title_direction', 'comment_direction'], axis=1)

In [9]:
df_test = pd.merge(df_x_test, dfs['activity_author'].groupby('activity_id').count().reset_index(),  
                  how='left', 
                  left_on='activity_id', right_on='activity_id', 
                  suffixes=('_x', 'activity_author'))

df_test = pd.merge(df_test, dfs['event'].groupby('run_id').count().reset_index(),  
                  how='left', 
                  left_on='run_id', right_on='run_id', 
                  suffixes=('_x', '_event'))

df_test = pd.merge(df_test, dfs['user_role'].drop_duplicates('user_id'),
                  how='left', 
                  left_on='user_id', right_on='user_id', 
                  suffixes=('_x', '_event'))

print(df_tmp.columns.values)
print(df_test.columns.values)

['id_bet' 'run_id' 'user_id' 'direction_id' 'activity_id' 'size_max'
 'size_min' 'is_educational' 'is_checkin_required' 'activity_type_id'
 'main_competence_id' 'author_id' 'id' 'place_id' 'time_slot_id' 'role_id']
['id_bet' 'run_id' 'user_id' 'direction_id' 'activity_id' 'size_max'
 'size_min' 'is_educational' 'is_checkin_required' 'activity_type_id'
 'main_competence_id' 'author_id' 'id' 'place_id' 'time_slot_id' 'role_id']


In [10]:
df_tmp = df_tmp.fillna(-1)
df_test = df_test.fillna(-1)

In [11]:
x_train = np.array(df_tmp)
x_test = np.array(df_test)
y_train = np.array(y)

In [12]:
x_train = x_train[[True if x < np.quantile(y_train, 0.95) else False for x in y_train]]
y_train = [x for x in y_train if x < np.quantile(y_train, 0.95)]

In [13]:
#Fitting
regr = xgb.XGBRegressor(n_estimators=68)
regr.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=68,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [14]:
#Cross validation score
regr_score(x_train, y_train, regr, rmsle)

[1.48561553 1.73304101 1.42651735 1.6966553  1.8745859 ]
mean score -- 1.6432830170522217


In [15]:
%%time
test_pred = regr.predict(x_test)

CPU times: user 6.09 ms, sys: 1.52 ms, total: 7.61 ms
Wall time: 5.89 ms


In [16]:
#First submit
submit = pd.concat([df_test['id_bet'], pd.Series(test_pred)], axis=1)
submit.columns=['id_bet', 'label']
submit.to_csv('submit_baseline.csv', index=False)

In [17]:
#Let's play with XGBoost parameters
#Fitting
regr = xgb.XGBRegressor(n_estimators=23, learning_rate = 0.1, max_depth = 4)
regr.fit(x_train, y_train)
#Cross validation score
regr_score(x_train, y_train, regr, rmsle)

[1.39051285 1.30915951 1.5058642  1.69237008 1.80835085]
mean score -- 1.5412514976801632


In [18]:
#Third submit with tuned parameters
submit = pd.concat([df_test['id_bet'], pd.Series(test_pred)], axis=1)
submit.columns=['id_bet', 'label']
submit.to_csv('submit_baseline.csv', index=False)

Попробуем сделать фит только на отобранных градиентным бустингом признаках

In [ ]:
df_feature = df_tmp.drop(['run_id'], axis=1, inplace=True)
df_feature = df_tmp.iloc[:, :2]
df_feature.head()

In [ ]:
df_test_feature = df_test.drop(['run_id'], axis=1, inplace=True)
df_test_feature = df_test.iloc[:, :2]
df_test_feature.head()

In [ ]:
x_train = np.array(df_feature)
x_test = np.array(df_test_feature)
y_train = np.array(y)

In [ ]:
x_train = x_train[[True if x < np.quantile(y_train, 0.95) else False for x in y_train]]
y_train = [x for x in y_train if x < np.quantile(y_train, 0.95)]

In [ ]:
#Fitting
regr = xgb.XGBRegressor(n_estimators=23, learning_rate = 0.1, max_depth = 4)
regr.fit(x_train, y_train)

In [ ]:
#Cross validation score
regr_score(x_train, y_train, regr, rmsle)

Ошибка уменьшилась!

In [ ]:
test_pred = regr.predict(x_test)

In [ ]:
#Second submit with selected features
submit = pd.concat([df_test['id_bet'], pd.Series(test_pred)], axis=1)
submit.columns=['id_bet', 'label']
submit.to_csv('submit_baseline.csv', index=False)

А что же получается для k - neighbors?

In [ ]:
#Fitting
regr = KNeighborsRegressor(n_neighbors=4, weights='distance',  p=1)
regr.fit(x_train, y_train)

In [ ]:
#Cross validation score
regr_score(x_train, y_train, regr, rmsle)

А для k-neighbors не сработало - ошибка чуть ухудшилась

Попробуем оттюнить параметры для градиентного бустинга с помощью random search для отобранных фичей

In [ ]:
from xgboost.sklearn import XGBRegressor  
import scipy.stats as st
seed = 342
np.random.seed(seed)
one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

params = {  
    "n_estimators": st.randint(3, 40),
    "max_depth": st.randint(3, 40),
    "learning_rate": st.uniform(0.05, 0.4),
    "colsample_bytree": one_to_left,
    "subsample": one_to_left,
    "gamma": st.uniform(0, 10),
    'reg_alpha': from_zero_positive,
    "min_child_weight": from_zero_positive,
}

xgbreg = XGBRegressor(nthreads=-1, seed=seed)  

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(xgbreg, params, scoring = rmsle, n_jobs=1, random_state = seed)  
gs.fit(x_train, y_train)  
gs.best_params_ 

In [ ]:
gs.best_score_

In [ ]:
regr = xgb.XGBRegressor(colsample_bytree = 0.97, n_estimators=23, reg_alpha = 27.21, 
                        subsample = 0.908, min_child_weight = 3.89, max_depth = 24, learning_rate = 0.26, gamma = 5.57)
regr.fit(x_train, y_train)
regr_score(x_train, y_train, regr, rmsle)